In [1]:
import numpy as np
import pandas as pd
import text_representation as repre
import helpers
import preprocessing as preproc
import time

#### Preprocessing

In [ ]:
preproc.do_preprocessing('../twitter-datasets/train_pos.txt')
preproc.do_preprocessing('../twitter-datasets/train_neg.txt')

In [2]:
lines, y = preproc.return_processed_trainset_and_y(False)

#### Text Representation

In [3]:
start_time = time.time()

In [ ]:
embeddings = repre.load_embeddings()
vocab_dict = repre.create_dict_from_provided_vocabulary()

In [ ]:
train_pos_path = '../twitter-datasets/train_pos_processed.txt'
pos_features = repre.create_tweet_features(train_pos_path, vocab_dict, shape_of_word_embeddings=20)

In [ ]:
train_neg_path = '../twitter-datasets/train_neg_processed.txt'
neg_features = repre.create_tweet_features(train_neg_path, vocab_dict, shape_of_word_embeddings=20)

In [ ]:
pos_features = pos_features[1:]
neg_features = neg_features[1:]

In [ ]:
pos_shape = pos_features.shape[0]
neg_shape = neg_features.shape[0]

In [ ]:
X = np.vstack((pos_features, neg_features))
y = np.zeros(shape=(pos_features.shape[0] + neg_features.shape[0]))
y[:pos_shape] = 1
y[pos_shape:] = -1

In [ ]:
np.unique(y)

In [ ]:
elapsed_time = divmod(round((time.time() - start_time)), 60)
print('------\nElapsed time: {m} min {s} sec\n'.format(m=elapsed_time[0], s=elapsed_time[1]))

#### Predictions

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(lines)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold

In [ ]:
clf = LogisticRegression().fit(X, y)

In [ ]:
# rs = ShuffleSplit(n_splits=4, test_size=.25, random_state=0)
kf = KFold(n_splits=4, shuffle=True, random_state=0)

In [ ]:
scores = cross_val_score(clf, X, y, cv=kf)
print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

###### LOAD TEST SET AND PREDICT

In [ ]:
preproc.do_preprocessing('../twitter-datasets/test_data.txt', test_file=True)

In [ ]:
test_processed_path = '../twitter-datasets/test_data_processed.txt'
test_features = repre.create_test_tweet_features(test_processed_path, vocab_dict, shape_of_word_embeddings=20)

In [ ]:
test_features = test_features[1:]
print(test_features.shape)

In [ ]:
y_pred = clf.predict(test_features)

In [ ]:
helpers.create_submission_csv(y_pred)

In [ ]:
np.unique(y_pred)